In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import pandas as pd
import json
from tqdm import tqdm
from datasets import dataset_dict, load_dataset, Dataset, DatasetDict
import random
import threading
import re

In [ ]:
default_path= '/content/drive/MyDrive/aihub/'
data = [d for d in os.listdir(default_path) if d != '.DS_Store' and d != 'Save']
print(data)

['문서요약 텍스트', '.ipynb_checkpoints', '018.논문자료 요약 데이터', '048.일반상식 문장 생성 데이터']


In [ ]:
default = default_path + '/문서요약 텍스트/Training'
default
data_foler = os.listdir(default)

In [ ]:
default = default_path + '/문서요약 텍스트/Training'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['input', 'instruction', 'output'])

text_list = []
instruction_list = []
for d in data_folder:
    path = default + '/' + d
    sub_df = pd.DataFrame(columns=['input', 'instruction', 'output'])
    print(path)

    with open(path, 'r', encoding='UTF8') as file:
      json_file = json.load(file)

    documents = json_file['documents']

    for i in tqdm(range(len(documents))):
      title = documents[i]['title']
      text = documents[i]['abstractive'][0]
      sentence_list = documents[i]['text']

      number = random.choice([0,1])

      # 난수가 발생하지 않은 경우 내용 요약만 진행
      if number == 0:
        instruction_text ='주어진 문장을 적절하게 요약해주세요.\n\n문장: '
        for sentence in sentence_list:
          try:
            if len(sentence) > 1:
              for sub in sentence:
                sen = sub['sentence']
                instruction_text += sen + ' '
            else:
              sen = sentence[0]['sentence']
              instruction_text += sen + ' '
          except:
            pass
        instruction_list.append(instruction_text.strip())
        text_list.append(documents[i]['abstractive'][0])

      # 난수가 발생한 경우 제목 및 내용 요약 진행
      else:
        instruction_text = '주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: '
        for sentenct in sentence_list:
          try:
            if len(sentenct) > 1:
              for sub in sentenct:
                sen = sub['sentence']
                instruction_text += sen + ' '
            else:
              sen = sentenct[0]['sentence']
              instruction_text += sen + ' '
          except:
            pass
          instruction_list.append(instruction_text.strip())
          answer = '제목: ' + title + '\n' + documents[i]['abstractive'][0]
          text_list.append(answer)

    sub_df['input'] = ''
    sub_df['instruction'] = instruction_list
    sub_df['output'] = text_list
    print(sub_df.iloc[0, 1])
    print(sub_df.iloc[0, 2])
    df = pd.concat([df, sub_df], axis=0)
    text_list = []
    instruction_list = []

/content/drive/MyDrive/aihub//문서요약 텍스트/Training/train_original.json


100%|██████████| 24329/24329 [00:00<00:00, 100875.69it/s]


주어진 문장을 적절하게 요약해주세요.

문장: 원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.
원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.
/content/drive/MyDrive/aihub//문서요약 텍스트/Training/train_original 3.json


100%|██████████| 243983/243983 [00:06<00:00, 40592.58it/s]


주어진 문장을 적절하게 요약해주세요.

문장: ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주

100%|██████████| 56760/56760 [00:02<00:00, 21112.39it/s]


주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.

문장: 이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 '공생발전'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다.
제목: [사설] 기부문화 확산 은근히 강조한 李대통령
이명박 대통령은 어제 30대 그룹 총수를 모아놓고 시대적 요구는 역시 총수가 앞장서야 한다고 발언하며 기부문화 확산을 은근히 강조했으나 대통령의 강요나 포퓰리즘에 의한 압박보다는 자발적 문화로 구축해야 효과가 더 큰 법으로 방안 마련을 맡기고 시간을 줘야 할 것으로 보인다.


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df['input'] = ''

In [ ]:
df

,input,instruction,output
0,,주어진 문장을 적절하게 요약해주세요.\n\n문장: 원고가 소속회사의 노동조합에서 분...,원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...
1,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 수출입업체...",제목: 법인세부과처분취소\n수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 ...
2,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 가등기담보...",제목: 양도소득세부과처분취소\n가등기담보권자가 제소전 화해조항에 의해 자기 명의로 ...
3,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 가. 부가...",제목: 부가가치세부과처분취소\n부가가치세법 제22조 제3항 단서에 제1호와 제2호가...
4,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 가. 부가...",제목: 부가가치세부과처분취소\n부가가치세법 제22조 제3항 단서에 제1호와 제2호가...
...,...,...,...
1406914,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 우려했던 ...","제목: [사설] 日 경제보복 현실화, 脫일본 전략 급하다\n외교문제로 인한 일본의 ..."
1406915,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 우려했던 ...","제목: [사설] 日 경제보복 현실화, 脫일본 전략 급하다\n외교문제로 인한 일본의 ..."
1406916,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 우려했던 ...","제목: [사설] 日 경제보복 현실화, 脫일본 전략 급하다\n외교문제로 인한 일본의 ..."
1406917,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 우려했던 ...","제목: [사설] 日 경제보복 현실화, 脫일본 전략 급하다\n외교문제로 인한 일본의 ..."


In [ ]:
# 데이터 재사용을 위해 parquet 형식으로 저장해서 관리함
df.to_parquet(default_path + '/Save/summary_com.parquet', engine='pyarrow', index=False)

In [ ]:
import random

# default_path = '/content/drive/MyDrive/gdrive/'
default = default_path + '048.일반상식 문장 생성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터'
default
data_folder = os.listdir(default)

df = pd.DataFrame(columns =['text', 'doc_id', 'domain'])

instruction_list = []
text_list = []
doc_id_list = []
doc_id = 0


for f in tqdm(data_folder):
  if f != ".DS_Store":
    print(f)
    path = default + '/' + f

    sub_df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

    try:
      with open(path, 'r', encoding='UTF8') as file:
        json_file = json.load(file)

      for documents in json_file:
        text = documents['sentence']
        genSentences = documents['genSentences']
        genChoice = random.choice(genSentences)
        text_list.append(genChoice['label-scenes'].strip())

        instruction_sentence = '임의의 순서대로 나열된 단어들을 보고 적절한 문장으로 재구성하세요.\n\n임의의 순서로 나열된 단어: ['
        for concept in documents['concepts']:
          instruction_sentence += concept['stem'] + ' '
        instruction_sentence = instruction_sentence.strip()[:-1]
        instruction_sentence += ']'

        instruction_list.append(instruction_sentence)
        doc_id_list.append('AIHUB_일반상식문장생성데이터_' +str(doc_id))
        doc_id += 1
    except Exception as e:
      print(e)
      pass

sub_df['domain'] = ''
sub_df['doc_id'] = doc_id_list
sub_df['text'] = text_list
df = pd.concat([df, sub_df], axis=0)
text_list = []
instruction_list = []
doc_id_list = []

  0%|          | 0/2 [00:00<?, ?it/s]

nia_2022_15-2_commonsense_TL.json


100%|██████████| 2/2 [00:24<00:00, 12.06s/it]


In [ ]:
df.to_parquet(default_path+'Save/일반상식문장생성데이터.parquet', engine = 'pyarrow', index = False)

In [ ]:
import pandas as pd
from datasets import dataset_dict, load_dataset, Dataset, DatasetDict
from tqdm import tqdm
import random
import string
import re

In [ ]:
new_df=pd.read_parquet(default_path + '/Save/일반상식문장생성데이터.parquet')

In [ ]:
filtered_df = new_df

In [ ]:
new_df

,text,doc_id,domain
0,아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.,AIHUB_일반상식문장생성데이터_0,None
1,마을은 이번에 열린 축제를 구경하는 사람들로 붐빈다.,AIHUB_일반상식문장생성데이터_1,None
2,그들은 우리의 시각을 자극하는 요소들을 정비했다.,AIHUB_일반상식문장생성데이터_2,None
3,나는 대학 시절부터 오늘날까지 이 책을 꾸준히 읽어 왔다.,AIHUB_일반상식문장생성데이터_3,None
4,농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.,AIHUB_일반상식문장생성데이터_4,None
...,...,...,...
406491,그들은 그에게 일제히 총격을 가해 왔다.,AIHUB_일반상식문장생성데이터_406491,None
406492,그가 직장을 그만두고 나서 힘들게 생활하고 있다.,AIHUB_일반상식문장생성데이터_406492,None
406493,그는 그 일을 아무 생각 없이 그대로 따라했다.,AIHUB_일반상식문장생성데이터_406493,None
406494,이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.,AIHUB_일반상식문장생성데이터_406494,None


In [ ]:
# parquet 파일을 텍스트 형식으로 변환
# 전처리 1
from typing import Text

# 구두점이 있는 문장 합치기
# for i in range(len(filtered_df)):
for i in range(1000):
    # 공백 제거
    filtered_df['text'][i] = filtered_df['text'][i].strip()

    try:
        # 마침표 다음 스페이스 -> 문장 하나
        if filtered_df['text'][i].endswith('. '):
            filtered_df['text'][i]=filtered_df['Text'][i][:-2]
        elif filtered_df['text'][i].endswith('.'):
            pass
        elif filtered_df['text'][i].endswith(' .'):
            print(i, '번째 문장')
            print(filtered_df['text'[i]])
            filtered_df['text'][i] = filtered_df['text'][i][:-2]+'.'

            print(filtered_df['text'][i])
        else:
            print(i, '번째 문장')
            print(filtered_df['text'][i])

            # 가장 끝에 공백 -> 문장이 완료된 상태이므로 구두점(.) 추가
            if filtered_df['text'][i].endswith(' '):
                filtered_df['text'][i] = filtered_df['text'][i][:-1]+'.'
            else:
                filtered_df['text'][i] = filtered_df['text'][i]+'.'

            print(filtered_df['text'][i])

    except Exception as e:
        print(e)
        print(i)
        print(filtered_df['text'][i])

        print('################################################')



<ipython-input-13-3cafbe8fe382>:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  filtered_df['text'][i] = filtered_df['text'][i].strip()


In [ ]:
# 전처리 2 - 특수문자 제거
remove_index = []
for i in range(len(filtered_df)):
    text = filtered_df['text'][i]
    if '�' in text:
        remove_index.append(i)
    elif '삭제.' in text[-5:]:
        remove_index.append(i)

In [ ]:
# 행삭제 및 인덱스 정리
filtered_df.drop(remove_index, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df

,text,doc_id,domain
0,아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.,AIHUB_일반상식문장생성데이터_0,None
1,마을은 이번에 열린 축제를 구경하는 사람들로 붐빈다.,AIHUB_일반상식문장생성데이터_1,None
2,그들은 우리의 시각을 자극하는 요소들을 정비했다.,AIHUB_일반상식문장생성데이터_2,None
3,나는 대학 시절부터 오늘날까지 이 책을 꾸준히 읽어 왔다.,AIHUB_일반상식문장생성데이터_3,None
4,농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.,AIHUB_일반상식문장생성데이터_4,None
...,...,...,...
406491,그들은 그에게 일제히 총격을 가해 왔다.,AIHUB_일반상식문장생성데이터_406491,None
406492,그가 직장을 그만두고 나서 힘들게 생활하고 있다.,AIHUB_일반상식문장생성데이터_406492,None
406493,그는 그 일을 아무 생각 없이 그대로 따라했다.,AIHUB_일반상식문장생성데이터_406493,None
406494,이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.,AIHUB_일반상식문장생성데이터_406494,None


In [ ]:
# 데이터 전처리 3 - 결측지 제거 : text 필드에 null인 행 제거
filtered_df=filtered_df.dropna(subset=['text'])

In [ ]:
filtered_df=filtered_df.reset_index(drop=True)

In [ ]:
def cal_element(text):
    return len(text.split('. '))

In [ ]:
filtered_df['len_elements'] = filtered_df['text'].apply(lambda x: len(x.split('. ')) if isinstance(x, str) else 1)


In [ ]:
filtered_df=filtered_df.reset_index(drop=True)

In [ ]:
filtered_df_1=filtered_df[(filtered_df['len_elements'] >= 2)&(filtered_df['len_elements'] < 15 )] # 2이상 15인 경우 카운팅
filtered_df_2=filtered_df[filtered_df['len_elements'] < 2] # 정상 문장 카운팅

In [ ]:
filtered_df_1=filtered_df_1.reset_index(drop=True)

In [ ]:
filtered_df['len_elements'].value_counts()

,count
len_elements,
1,406494
2,2


In [ ]:
filtered_df_1

,text,doc_id,domain,len_elements
0,하나에 열중하니 아무래도. 다른 것에는 소원하게 되었다.,AIHUB_일반상식문장생성데이터_101832,None,2
1,"검은색, 붉은색. 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.",AIHUB_일반상식문장생성데이터_191123,None,2


In [ ]:
filtered_df_1['text'] = ['하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다.', '검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.']

In [ ]:
filtered_df_1['len_elements'] = 1

In [ ]:
filtered_df_1

,text,doc_id,domain,len_elements
0,"하나에 열중하니 아무래도, 다른 것에는 소원하게 되었다.",AIHUB_일반상식문장생성데이터_101832,None,1
1,"검은색, 붉은색, 녹색의 다채로운 쌀과, 찹쌀 등을 생산하고 있다.",AIHUB_일반상식문장생성데이터_191123,None,1


In [ ]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

In [ ]:
filtered_df_2 = pd.concat([filtered_df_2, filtered_df_1], axis=0)

In [ ]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

In [ ]:
# order inference를 진행하는 함수
def make_text_allign_data(text):
    if not text.endswith('.') or text.endswit('. '):
        text += '.'

    if text.endswith('.'):
        text+=' '

    my_list = text.split('. ')
    my_list_with_dots = [element + '.' for element in my_list]

    if my_list_with_dots[-1] == '.':
        my_list_with_dots=my_list_with_dots[:-1]

    random.shuffle(my_list_with_dots)

    if len(my_list_with_dots) == 1:
        raise Exception("allign 리스트 기이가 1입니다.", text)


    tmp_input='당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.'
    tmp_inst=f'이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: {my_list_with_dots}'

    if text.endswith(' '):
        tmp_out=text[:-1]
    else:
        tmp_out=text

    return tmp_input, tmp_inst, tmp_out


In [ ]:
# 문장이 주어졌을때 마지막 문장 예측하는 함수
def make_completion_data(text):
  if not (text.endswith('.') or text.endswith('. ')):
    text += '.'

  if text.endswith('. '):
    text=text
  else:
    text = text +' '

  my_list_with_dots = text.split('. ')

  if my_list_with_dots[-1] == '':
    my_list_with_dots=my_list_with_dots[:-1]

    if len(my_list_with_dots) ==1:
      raise Exception("Completions 텍스트를 나눈 리스트의 길이가 1입니다.", text)

    last_sentence = my_list_with_dots.pop()
    last_sentence += '.'

  remaining_paragraph=''

  for i in range(len(my_list_with_dots)):
    if i!=len(my_list_with_dots)-1:
                 remaining_paragraph += my_list_with_dots[i]+'. '
    else:
      remaining_paragraph += my_list_with_dots[i]+'.' # 문장을 기억하고 있도록 설정

  tmp_input='당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'
  tmp_instruct=f'다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.\n#텍스트: {remaining_paragraph}'
  tmp_output=last_sentence

  return tmp_input, tmp_instruct, tmp_output

In [ ]:
# 괄호 안에 문장이 들어도록
def make_text_mask_data(text):
  if not (text.endswith('.') or text.endswith('. ')):
    text += '.'

  if text.endswith('. '):
    text=text[:-1]
  else:
    text = text

  words = re.findall(r'[가-힣]{2,}', text)
  random_word = random.choice(words)

  masked_text = text.replace(random_word, '<MASK>')

  tmp_input='당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.'
  tmp_instruct=f'이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를 예측해야 합니다. <MASK>가 위치한 문장의 전체 문맥을 분석하여, 문장의 나머지 내용과 일관되게 <MASK>에 들어갈 가장 적합한 단어를 답하세요.\n#텍스트: {masked_text}'
  tmp_output=random_word

  return tmp_input, tmp_instruct, tmp_output

In [ ]:
def make_word_align(text):
  word_lst=[]
  for word in text.split(' '):
    out = re.sub(r"[^\w\s]", "", word)
    word_lst.append(out)

  #중복 제거하기 위해 set으로 만듬
  word_lst = set(word_lst)

  #다시 리스트 형식으로 돌림
  word_lst=list(word_lst)

  #랜덤하게 재배열
  random.shuffle(word_lst)

  tmp_input='당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.'
  tmp_instruct=f'이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들이 담긴 리스트가 제공됩니다. 이 리스트의 단어를 모두 활용하여 가장 문맥상 적절한 문장을 생성하세요.\n#단어리스트: {word_lst}'
  tmp_output=text

  return tmp_input, tmp_instruct, tmp_output

In [41]:
# 데이터 생성 파이프라인
from os import TMP_MAX
input_lst=[]
output_lst=[]
inst_lst=[]
id_lst=[]

for i in tqdm(range(len(filtered_df_2))):
  try:
    text=filtered_df_2['text'][i]

    thred=random.random()

    if thred < 0.56:
      TMP_MAXtmp_id='word_align_aihub'
    else:
      tmp_input, tmp_instruct, tmp_output = make_text_mask_data(text)
      # tmp_input, tmp_instruct, tmp_output = make_completion_data(text)
      tmp_id='pre_mask_aihub'

    input_lst.append(tmp_input)
    inst_lst.append(tmp_instruct)
    output_lst.append(tmp_output)
    id_lst.append(tmp_id)

  except Exception as e:
    print(f"{i}번째 행")
    print(e)
    print(filtered_df_2['text'][i])
    print("---------------------------------------------")

  6%|▋         | 26110/406496 [00:00<00:02, 130892.04it/s]

0번째 행
name 'tmp_input' is not defined
아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.
---------------------------------------------


100%|██████████| 406496/406496 [00:05<00:00, 79992.87it/s]


In [42]:
hub_df_2=pd.DataFrame({'input':input_lst,'instruction':inst_lst,'output':output_lst})

In [43]:
hub_df = hub_df_2.copy()

In [44]:
hub_df=hub_df.reset_index(drop=True)

In [45]:
hub_df

,input,instruction,output
0,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열린
1,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열린
2,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열린
3,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열린
4,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열린
...,...,...,...
406490,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,일제히
406491,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,일제히
406492,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,옷이
406493,당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.,이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를...,열중하니


In [47]:
hub_df.to_parquet(default_path+'Save/일반상식문장생성데이터2.parquet', engine = 'pyarrow', index = False)

In [48]:
! pip install datasets
! pip install huggingface_hub

In [49]:
from huggingface_hub import login
from datasets import Dataset
login()

In [50]:
dataset = Dataset.from_pandas(hub_df)

In [52]:
dataset.push_to_hub('wonik-hi/korea_common_sence')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/407 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wonik-hi/korea_common_sence/commit/47791bc3f06092126b518a72c1667cc41e553c86', commit_message='Upload dataset', commit_description='', oid='47791bc3f06092126b518a72c1667cc41e553c86', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/wonik-hi/korea_common_sence', endpoint='https://huggingface.co', repo_type='dataset', repo_id='wonik-hi/korea_common_sence'), pr_revision=None, pr_num=None)